# Homework 9: Question 13.2

In [1]:
# Dependencies
import simpy
import random
import numpy as np

__Setting up simulation variables__

In [2]:
class Queue(object):
    
    def __init__(self, env, num_queue, queue_time):
        self.env = env
        
        # Create 'num_queue' Queues that take 'queue_time' amount of time
        self.queue = simpy.Resource(env, num_queue)
        self.queue_time = queue_time
    
    
    def checkin(self, passenger):
        # it takes 'queueTime' time for a passenger to finish the queue
        yield self.env.timeout(self.queue_time)
        

In [3]:
arrivalTime = []
queueTime = []
def passenger(env, name, queue):
    
    arrivalTime.append(env.now)
    print('%s enters the queue at %.2f.' % (name, env.now))
    
    with queue.queue.request() as request:
        yield request
        
        # QUEUE
        yield env.process(queue.checkin(name))
        queueTime.append(env.now)
        
        
        print('%s leaves the queue at %.2f.' % (name, env.now))

In [4]:
# numQueue and time go to AirPort
# interval is local in the user-defined-function
def setup(env, num_queue, queue_time, interval):
    
    # Create the AirPort with 5 queues taking 0.75 minutes each
    queue = Queue(env, num_queue, queue_time)
    
    # Create 1 initial passenger
    i=1
    env.process(passenger(env, 'Passenger %i' % i, queue))
    
    # Create more passengers while simulation is running
    while True:
        yield env.timeout(interval)
        i += 1
        env.process(passenger(env, 'Passenger %i' % i, queue))

In [5]:
random.seed(42)
env = simpy.Environment()
env.process(setup(env, 1, 0.75, 0.2))

env.run(until=15)

Passenger 1 enters the queue at 0.00.
Passenger 2 enters the queue at 0.20.
Passenger 3 enters the queue at 0.40.
Passenger 4 enters the queue at 0.60.
Passenger 1 leaves the queue at 0.75.
Passenger 5 enters the queue at 0.80.
Passenger 6 enters the queue at 1.00.
Passenger 7 enters the queue at 1.20.
Passenger 8 enters the queue at 1.40.
Passenger 2 leaves the queue at 1.50.
Passenger 9 enters the queue at 1.60.
Passenger 10 enters the queue at 1.80.
Passenger 11 enters the queue at 2.00.
Passenger 12 enters the queue at 2.20.
Passenger 3 leaves the queue at 2.25.
Passenger 13 enters the queue at 2.40.
Passenger 14 enters the queue at 2.60.
Passenger 15 enters the queue at 2.80.
Passenger 4 leaves the queue at 3.00.
Passenger 16 enters the queue at 3.00.
Passenger 17 enters the queue at 3.20.
Passenger 18 enters the queue at 3.40.
Passenger 19 enters the queue at 3.60.
Passenger 5 leaves the queue at 3.75.
Passenger 20 enters the queue at 3.80.
Passenger 21 enters the queue at 4.00.


Basically, we now have the times that the passengers left the queue. We can now have passengers go to the scanner; but instead of an interval of 0.2, we will have them go to the scanner at each moment a passenger left the queue. These values are saved in a list, called queueTime.

In [6]:
# Exact same thing as Queue but with uniformly random scan time of 0.5 to 1
class Scan(object):
    def __init__(self, env, num_scan):
        self.env = env
        
        self.scan=simpy.Resource(env, num_scan)
        
    def scan(self, passenger):
        # Random value 0.5 to 1 (up to hundredth digit)
        scan_time = random.randint(50, 100)/100
        yield self.env.timeout(scan_time)

In [7]:
scanTime = []
def passenger2(env, name, scan):
    
    print('%s enters the queue at %.2f.' % (name, env.now))
    
    with scan.scan.request() as request:
        yield request
        
        # SCAN
        yield env.process(scan.scan(name))
        scanTime.append(env.now)
        
        
        print('%s leaves the queue at %.2f.' % (name, env.now))

In [8]:
# numQueue and time go to AirPort
# interval is local in the user-defined-function
def setup2(env, num_scan, queueTime):
    
    # Create the AirPort with 5 queues taking 0.75 minutes each
    scan = Scan(env, num_scan)
    
    # Create 1 initial passenger
    i=1
    env.process(passenger2(env, 'Passenger %i' % i, scan))
    
    # Create more passengers while simulation is running
    while True:
        yield env.timeout(queueTime[i])
        i += 1
        env.process(passenger2(env, 'Passenger %i' % i, scan))

In [9]:
env = simpy.Environment()
env.process(setup2(env, 1, queueTime))

env.run(until=100)

Passenger 1 enters the queue at 0.00.


TypeError: 'Resource' object is not callable